In [1]:
import json
import os

from omegaconf import DictConfig, OmegaConf

from nemo.collections.nlp.models import ZeroShotIntentModel
from nemo.core.config import hydra_runner
from nemo.utils import logging
from nemo.collections import nlp as nemo_nlp

import pandas as pd
import numpy as np
import faiss

[NeMo W 2022-03-07 13:14:35 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    
[NeMo W 2022-03-07 13:14:35 experimental:27] Module <function get_argmin_mat at 0x7f77c09af280> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-03-07 13:14:35 experimental:27] Module <function getMultiScaleCosAffinityMatrix at 0x7f77c09af310> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-03-07 13:14:35 experimental:27] Module <function parse_scale_configs at 0x7f77c09afe50> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-03-07 13:14:35 experimental:27] Module <function get_embs_and_timestamps at 0x7f77c09afee0> is experim

In [2]:
# BC7 Track3 data dir containing training and val datasets
bc7_tr3_datadir = '/datasets/bc7/Track-3_Med-Tweets'
# pre-trained NLI model on NGC
pretrained_model = nemo_nlp.models.ZeroShotIntentModel.from_pretrained("zeroshotintent_en_bert_base_uncased")

[NeMo I 2022-03-07 13:14:36 cloud:56] Found existing object /root/.cache/torch/NeMo/NeMo_1.6.0/zeroshotintent_en_bert_base_uncased/bd0b8d430477bc6e329c79219ca77690/zeroshotintent_en_bert_base_uncased.nemo.
[NeMo I 2022-03-07 13:14:36 cloud:62] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.6.0/zeroshotintent_en_bert_base_uncased/bd0b8d430477bc6e329c79219ca77690/zeroshotintent_en_bert_base_uncased.nemo
[NeMo I 2022-03-07 13:14:36 common:729] Instantiating model from pre-trained checkpoint
[NeMo I 2022-03-07 13:14:42 tokenizer_utils:125] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-base-uncased, vocab_file: /tmp/tmppw1iee4l/9cf58d5590404f6ca6ffa00bbc4b1ac5_tokenizer.vocab_file, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
[NeMo W 2022-03-07 13:14:43 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    file_name: train.tsv
    shuffle: true
    num_samples: -1
    batch_size: 32
    
[NeMo W 2022-03-07 13:14:43 modelPT:149] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    file_name: dev_matched.tsv
    shuffle: false
    num_samples: -1
    batch_size: 32
    
[NeMo W 2022-03-07 13:14:43 modelPT:1216] World size can only be set by PyTorch Lightning Trainer.
[NeMo W 2022-03-07 13:14:43 modelPT:209] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for 

[NeMo I 2022-03-07 13:14:49 save_restore_connector:154] Model ZeroShotIntentModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.6.0/zeroshotintent_en_bert_base_uncased/bd0b8d430477bc6e329c79219ca77690/zeroshotintent_en_bert_base_uncased.nemo.


In [3]:
candidate_labels = []

tr_df_0 = pd.read_csv(os.path.join(bc7_tr3_datadir, 'BioCreative_TrainTask3.0.tsv'), sep='\t')
tr_df_1 = pd.read_csv(os.path.join(bc7_tr3_datadir, 'BioCreative_TrainTask3.1.tsv'), sep='\t')
df_tr = pd.concat([tr_df_0, tr_df_1])
df_val = pd.read_csv(os.path.join(bc7_tr3_datadir, 'BioCreative_ValTask3_corrected.tsv'), sep='\t')

In [4]:
# Tweets in the training set that contain Drugs
tr_tweets = df_tr['text'].to_list()
tr_drugs = df_tr['drug'].to_list()

In [5]:
predictions = pretrained_model.predict(tr_tweets, candidate_labels, batch_size=32, multi_label=True)

[NeMo I 2022-03-07 13:14:49 zero_shot_intent_dataset:234] Processing queries for inference
[NeMo I 2022-03-07 13:14:49 glue_benchmark_dataset:237] Writing example 0 of 89004
[NeMo I 2022-03-07 13:14:49 glue_benchmark_dataset:319] *** Example ***
[NeMo I 2022-03-07 13:14:49 glue_benchmark_dataset:320] guid: query-0-label-0
[NeMo I 2022-03-07 13:14:49 glue_benchmark_dataset:321] tokens: [CLS] @ r ##hy _ q ##d ##10 yeah ir ##king he need his ass who ##op i really don ' t feel good i swear i ' m about to go to the hospital [SEP]
[NeMo I 2022-03-07 13:14:49 glue_benchmark_dataset:322] input_ids: 101 1030 1054 10536 1035 1053 2094 10790 3398 20868 6834 2002 2342 2010 4632 2040 7361 1045 2428 2123 1005 1056 2514 2204 1045 8415 1045 1005 1049 2055 2000 2175 2000 1996 2902 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[NeMo I 2022-03-07 13:14:49 glue_benc

In [6]:
tr_tweet_embs = np.concatenate(predictions, axis=0)

In [7]:
val_tweets = df_val['text'].to_list()
val_drugs = df_val['drug'].to_list()

In [8]:
predictions = pretrained_model.predict(val_tweets, candidate_labels, batch_size=32, multi_label=True)

[NeMo I 2022-03-07 13:19:09 zero_shot_intent_dataset:234] Processing queries for inference
[NeMo I 2022-03-07 13:19:09 glue_benchmark_dataset:237] Writing example 0 of 38149
[NeMo I 2022-03-07 13:19:09 glue_benchmark_dataset:319] *** Example ***
[NeMo I 2022-03-07 13:19:09 glue_benchmark_dataset:320] guid: query-0-label-0
[NeMo I 2022-03-07 13:19:09 glue_benchmark_dataset:321] tokens: [CLS] i ' m 37 weeks pregnant so i can do whatever the hell i want [SEP]
[NeMo I 2022-03-07 13:19:09 glue_benchmark_dataset:322] input_ids: 101 1045 1005 1049 4261 3134 6875 2061 1045 2064 2079 3649 1996 3109 1045 2215 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[NeMo I 2022-03-07 13:19:09 glue_benchmark_dataset:323] input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [9]:
val_tweet_embs = np.concatenate(predictions, axis=0)

In [10]:
index = faiss.IndexFlatL2(tr_tweet_embs.shape[1])

In [11]:
print(index.is_trained)

True


In [12]:
index.add(tr_tweet_embs)

In [13]:
index.ntotal

89004

In [14]:
val_tweets_not_none = df_val[df_val['drug'] != '-']
val_tweets_none = df_val[df_val['drug'] == '-']

In [15]:
val_tweets_not_none_sample_idx = val_tweets_not_none.index[np.random.randint(0, high=len(val_tweets_not_none), size=5)]
val_tweets_none_sample_idx = val_tweets_none.index[np.random.randint(0, high=len(val_tweets_none), size=5)]

In [16]:
df_val.iloc[list(val_tweets_not_none_sample_idx) + list(val_tweets_none_sample_idx)]

,tweet_id,user_id,created_at,text,start,end,span,drug
36492,735212614989557760,431145629,2016-05-24,"Oaklee's heartbeat was 142 today,and I have fi...",91,97,Zantac,zantac
5909,529668542036447232,2444385272,2014-11-04,Too bad I'm terrified of the epidural,29,37,epidural,epidural
30701,677248150524919808,1465089974,2015-12-16,"@elizabeth81013 I hope I get orange, orange is...",114,122,epidural,epidural
20110,407452452133171200,1115191412,2013-12-02,@TSuntken I thought you were sleeping.. I can'...,76,93,sleeping medicine,sleeping medicine
25410,549455891331026944,1154971159,2014-12-29,Mommy caved &amp; got dilaudid .... Happy time...,22,30,dilaudid,dilaudid
10993,754921420908535808,252235856,2016-07-18,"Don't miss me when I'm gone, miss me before I ...",-,-,-,-
8798,752185087408545792,1959970807,2016-07-10,"@Morgan_Shyann3 thanks Morg! you too homegirl,...",-,-,-,-
14557,758777720562147332,19546372,2016-07-28,Heavy D? Does that stand for heavily deranged?...,-,-,-,-
24607,841651781390237697,583610519,2017-03-14,All I can ask is for you to realize where you ...,-,-,-,-
23700,538832434134282240,475534141,2014-11-29,Greg P. Courtney Jacob http://t.co/XXpOEzFvtD,-,-,-,-


In [37]:
k = 5
sample_list_idxs = list(val_tweets_not_none_sample_idx) + list(val_tweets_none_sample_idx)

In [29]:
D, I = index.search(val_tweet_embs[sample_list_idxs,:], k)

In [47]:
for i in range(len(sample_list_idxs)):
    print('Original Text: {}; Drug: {}'.format(df_val.iloc[sample_list_idxs[i]]['text'],
                                            df_val.iloc[sample_list_idxs[i]]['drug']))
    print('------')
    print('Retrieved Top-{}'.format(k))
    print('------')
    for j in range(k):
        print('{}: Text: {}; Drug: {}'.format(j, df_tr.iloc[I[i,j]]['text'], df_tr.iloc[I[i,j]]['drug']))
    print('======')

Original Text: Oaklee's heartbeat was 142 today,and I have finally managed to gain 3 pounds.Thank god for Zantac that keeps me from heartburn and puking😊💚; Drug: zantac
------
Retrieved Top-5
------
0: Text: @lilyfharris he couldn't poo, bless him. So he's having bowel flushes till we get the results back from his last test. He's ok though :) xx; Drug: -
1: Text: I guess I'm lucky that iv got away with feeling fine from 14-38 weeks but I really feel crap now! Enough? #yesplease; Drug: -
2: Text: I can go into labor next week &amp; my doctor said he will not stop me, meanwhile my hospital bags aren't packed😩; Drug: -
3: Text: @MyMdamer9 haha I'm jk. Well I did order 80$ worth but I took orders from BC's work mates. They ordered a bunch. But I got myself 8 boxes 😂; Drug: -
4: Text: Thank the lord for flexible working... 8 days left... 7 of those will be little days! #41daystilldday; Drug: -
Original Text: Too bad I'm terrified of the epidural; Drug: epidural
------
Retrieved Top-5
------